In [388]:
import pandas as pd

# Replace 'your_file.csv' with the path to your CSV file
file_path = '/Users/bruno/Documents/Statistical_Analyst_Products/combined_ratings.csv'

# Read the CSV file
df = pd.read_csv(file_path)
df['Time'] = pd.to_datetime(df['Review - Time'], unit='s').dt.strftime('%w-%m-%Y')
df['Rating First Digit'] = df['Review - Overall'].astype(str).str[0].astype(int)
df = df[df['File Name'].str.contains("RateBeer", case=False)]

# Print the first few rows of the DataFrame
print(df.head())

/var/folders/88/g84kk_9n2498jwvfh8mnxj8w0000gn/T/ipykernel_96284/2282718759.py:7: DtypeWarning: Columns (2,4,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


         Unnamed: 0                                Name Beer ID  Brewer ID  \
1586614           0            John Harvards Simcoe IPA   63836       8481   
1586615           1            John Harvards Simcoe IPA   63836       8481   
1586616           2       John Harvards Cristal Pilsner   71716       8481   
1586617           3  John Harvards Fancy Lawnmower Beer   64125       8481   
1586618           4  John Harvards Fancy Lawnmower Beer   64125       8481   

         ABV                    Beer Style Review - Appearance Review - Aroma  \
1586614  5.4  India Pale Ale &#40;IPA&#41;                 4/5           6/10   
1586615  5.4  India Pale Ale &#40;IPA&#41;                 4/5           6/10   
1586616    5             Bohemian Pilsener                 4/5           5/10   
1586617  5.4                        Kölsch                 2/5           4/10   
1586618  5.4                        Kölsch                 2/5           4/10   

        Review - Palate Review - Taste Revie

In [389]:
#print(len(df)) #4.5M using both Filenames
print(len(df)) #2.9 using only RateBeer

2924163


In [390]:
df = df.head(1000)
print(len(df)) #2.9 using only RateBeer
print(df.columns)

1000
Index(['Unnamed: 0', 'Name', 'Beer ID', 'Brewer ID', 'ABV', 'Beer Style',
       'Review - Appearance', 'Review - Aroma', 'Review - Palate',
       'Review - Taste', 'Review - Overall', 'Review - Time',
       'Review - Profile Name', 'Review - Text', 'File Name', 'Time',
       'Rating First Digit'],
      dtype='object')


In [391]:
def convert_to_decimal(review_str):
    try:
        numerator, denominator = map(float, review_str.split('/'))
        return numerator / denominator if denominator != 0 else 0
    except ValueError:
        # Handle cases where the review string is not in the expected format
        return None

# Apply this function to the 'Review - Overall' column
df['Review - Overall Decimal'] = df['Review - Overall'].apply(convert_to_decimal)

# Check the first few rows to verify the transformation
print(df[['Review - Overall', 'Review - Overall Decimal']].head())

        Review - Overall  Review - Overall Decimal
1586614            13/20                      0.65
1586615            13/20                      0.65
1586616            14/20                      0.70
1586617             8/20                      0.40
1586618             8/20                      0.40


In [392]:
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

In [393]:
########## BrewerID & aux variables
df3 = pd.DataFrame(df['Brewer ID'])
df3['Unnamed: 0'] = df['Unnamed: 0']
df3['Time'] = df['Time']
df3['Beer Style'] = df['Beer Style']
df3['Rating First Digit'] = df['Rating First Digit']
df3['Review - Overall'] = df['Review - Overall Decimal']
df3['Beer ID'] = df['Beer ID']
df3['ABV'] = df['ABV']


print(df3.head())
print(len(df3))

         Brewer ID  Unnamed: 0       Time                    Beer Style  \
1586614       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1586615       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
1586616       8481           2  5-05-2000             Bohemian Pilsener   
1586617       8481           3  4-09-2006                        Kölsch   
1586618       8481           4  4-09-2006                        Kölsch   

         Rating First Digit  Review - Overall Beer ID  ABV  
1586614                   1              0.65   63836  5.4  
1586615                   1              0.65   63836  5.4  
1586616                   1              0.70   71716    5  
1586617                   8              0.40   64125  5.4  
1586618                   8              0.40   64125  5.4  
1000


In [394]:
########## Time
# The format '%w-%m-%Y' represents the day of the week, month, and year
df3['Time'] = pd.to_datetime(df['Review - Time'], unit='s').dt.strftime('%w-%m-%Y')

print("column Time was sucesfully created")
print(df3.head())
print(len(df3))

column Time was sucesfully created
         Brewer ID  Unnamed: 0       Time                    Beer Style  \
1586614       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1586615       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
1586616       8481           2  5-05-2000             Bohemian Pilsener   
1586617       8481           3  4-09-2006                        Kölsch   
1586618       8481           4  4-09-2006                        Kölsch   

         Rating First Digit  Review - Overall Beer ID  ABV  
1586614                   1              0.65   63836  5.4  
1586615                   1              0.65   63836  5.4  
1586616                   1              0.70   71716    5  
1586617                   8              0.40   64125  5.4  
1586618                   8              0.40   64125  5.4  
1000


In [395]:
########## NewProduct Calculation
# Find the first review date for each product ('Beer ID') in the original DataFrame
first_review = df.groupby('Beer ID')['Review - Time'].min().reset_index()
print(first_review.head())

first_review['FirstReviewTime'] = pd.to_datetime(first_review['Review - Time'], unit='s').dt.strftime('%w-%m-%Y')

print(first_review.head())

# Merge the first review dates with the 'Beer ID', 'Brewer ID', and 'Beer Style' from the original DataFrame
# This associates each first review with its corresponding product and brewer
# 'drop_duplicates' is used to ensure each 'Beer ID' is only counted once
first_review = first_review.merge(df[['Beer ID', 'Brewer ID', 'Beer Style']], on='Beer ID', how='left').drop_duplicates('Beer ID')

print(first_review.head())
print(len(df3))

   Beer ID  Review - Time
0     1470      958694400
1     7721      990576000
2    19690     1058313600
3    19691     1058313600
4    19692     1058313600
   Beer ID  Review - Time FirstReviewTime
0     1470      958694400       5-05-2000
1     7721      990576000       3-05-2001
2    19690     1058313600       3-07-2003
3    19691     1058313600       3-07-2003
4    19692     1058313600       3-07-2003
    Beer ID  Review - Time FirstReviewTime  Brewer ID  \
0      1470      958694400       5-05-2000       8481   
1      7721      990576000       3-05-2001       8481   
2     19690     1058313600       3-07-2003       3228   
125   19691     1058313600       3-07-2003       3228   
158   19692     1058313600       3-07-2003       3228   

                          Beer Style  
0    Belgian White &#40;Witbier&#41;  
1                           Mild Ale  
2                          Brown Ale  
125                American Pale Ale  
158                           Porter  
1000


In [396]:
# Count new products for each 'Brewer ID' in each time period
# This gives the number of new products introduced by each brewer in each time period
new_product_count = first_review.groupby(['Brewer ID', 'FirstReviewTime']).size().reset_index(name='NewProduct')
print("aca")
print(new_product_count.head())

aca
   Brewer ID FirstReviewTime  NewProduct
0       3228       0-01-2009           1
1       3228       0-08-2010           1
2       3228       0-09-2010           1
3       3228       1-02-2006           1
4       3228       1-03-2009           1


In [397]:
# Merge the new product count with df3, matching on both 'BrewerID' and 'Time'
df3 = df3.merge(new_product_count, left_on=['Brewer ID', 'Time'], right_on=['Brewer ID', 'FirstReviewTime'], how='left')

# Drop unnecessary columns and handle missing data by filling NaNs with 0
df3['NewProduct'].fillna(0, inplace=True)
df3['NewProduct'] = df3['NewProduct'].apply(lambda x: 1 if x != 0 else 0)

print("column NewProduct was sucesfully created")
print(df3.head())

column NewProduct was sucesfully created
   Brewer ID  Unnamed: 0       Time                    Beer Style  \
0       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
2       8481           2  5-05-2000             Bohemian Pilsener   
3       8481           3  4-09-2006                        Kölsch   
4       8481           4  4-09-2006                        Kölsch   

   Rating First Digit  Review - Overall Beer ID  ABV FirstReviewTime  \
0                   1              0.65   63836  5.4       4-09-2006   
1                   1              0.65   63836  5.4       0-09-2006   
2                   1              0.70   71716    5       5-05-2000   
3                   8              0.40   64125  5.4       4-09-2006   
4                   8              0.40   64125  5.4       4-09-2006   

   NewProduct  
0           1  
1           1  
2           1  
3           1  
4           1  


In [398]:
print(df3.head())
print(len(df3))

   Brewer ID  Unnamed: 0       Time                    Beer Style  \
0       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
2       8481           2  5-05-2000             Bohemian Pilsener   
3       8481           3  4-09-2006                        Kölsch   
4       8481           4  4-09-2006                        Kölsch   

   Rating First Digit  Review - Overall Beer ID  ABV FirstReviewTime  \
0                   1              0.65   63836  5.4       4-09-2006   
1                   1              0.65   63836  5.4       0-09-2006   
2                   1              0.70   71716    5       5-05-2000   
3                   8              0.40   64125  5.4       4-09-2006   
4                   8              0.40   64125  5.4       4-09-2006   

   NewProduct  
0           1  
1           1  
2           1  
3           1  
4           1  
1000


In [399]:
########## NewMarket Calculation
# For each 'BeerID', determine if its first review is also the first entry of the brewer into that 'Beer Style'
# This step identifies first market entries into new beer styles for each brewer
first_style_entry = first_review.drop_duplicates(subset=['Brewer ID', 'Beer Style'])

# Count these first entries for each 'Brewer ID' and time period
# This gives the number of new market segments (beer styles) each brewer entered in each time period
new_market_count = first_style_entry.groupby(['Brewer ID', 'FirstReviewTime']).size().reset_index(name='NewMarket')

# Merge the new market count with df3, matching on both 'BrewerID' and 'Time'
df3 = df3.merge(new_market_count, left_on=['Brewer ID', 'Time'], right_on=['Brewer ID', 'FirstReviewTime'], how='left')

# Drop unnecessary columns and handle missing data by filling NaNs with 0
df3['NewMarket'].fillna(0, inplace=True)

df3['NewMarket'] = df3['NewMarket'].apply(lambda x: 1 if x != 0 else 0)
df3.drop(columns=['FirstReviewTime_x'], inplace=True)
df3 = df3.rename(columns={'FirstReviewTime_y': 'FirstReviewTime'})



In [400]:
print(df3.head())
print(len(df3))

   Brewer ID  Unnamed: 0       Time                    Beer Style  \
0       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
2       8481           2  5-05-2000             Bohemian Pilsener   
3       8481           3  4-09-2006                        Kölsch   
4       8481           4  4-09-2006                        Kölsch   

   Rating First Digit  Review - Overall Beer ID  ABV  NewProduct  \
0                   1              0.65   63836  5.4           1   
1                   1              0.65   63836  5.4           1   
2                   1              0.70   71716    5           1   
3                   8              0.40   64125  5.4           1   
4                   8              0.40   64125  5.4           1   

  FirstReviewTime  NewMarket  
0       4-09-2006          1  
1       0-09-2006          1  
2       5-05-2000          1  
3       4-09-2006          1  
4       4-09-2006    

In [401]:
########## NewCategory Calculation
# Identify the first occurrence of each BeerStyle
first_style_occurrence = df.groupby('Beer Style')['Review - Time'].min().reset_index()
first_style_occurrence['FirstStyleTime'] = pd.to_datetime(first_style_occurrence['Review - Time'], unit='s').dt.strftime('%w-%m-%Y')

# Count new BeerStyle categories for each time period
# This gives the number of new beer styles introduced in each time period
new_category_count = first_style_occurrence.groupby('FirstStyleTime').size().reset_index(name='NewCategory')

# Merge this count with df3
df3 = df3.merge(new_category_count, left_on='Time', right_on='FirstStyleTime', how='left')

# Drop the extra FirstStyleTime column
df3.drop('FirstStyleTime', axis=1, inplace=True)

# Handle missing data (if there are time periods with no new categories)
df3['NewCategory'].fillna(0, inplace=True)
df3['NewCategory'] = df3['NewCategory'].apply(lambda x: 1 if x != 0 else 0)

print("column NewCategory was sucesfully created")

column NewCategory was sucesfully created


In [402]:
########## Imitation Calculation

# Filter df3 for new market entries
new_market_entries = df3[df3['NewMarket'] > 0]

# Adjust the grouping in `product_launches`
product_launches = new_market_entries.groupby(['Beer Style', 'Time', 'Brewer ID']).size().reset_index(name='NewMarketEntryCount')

# Define the total number of rows in the DataFrame
total_rows = len(product_launches)

# Set an interval for printing progress updates
print_interval = 100  # Modify this number as needed

def calculate_imitation(brewer_id, beer_style, time, product_launches_df):
    time_window_start = datetime.strptime(time, '%w-%m-%Y') - relativedelta(months=3)
    time_window_start_str = time_window_start.strftime('%w-%m-%Y')

    competitor_launches = product_launches_df[
        (product_launches_df['Beer Style'] == beer_style) & 
        (product_launches_df['Time'] >= time_window_start_str) &
        (product_launches_df['Time'] < time) &
        (product_launches_df['Brewer ID'] != brewer_id)
    ]
    return competitor_launches['NewMarketEntryCount'].sum()

# Create a column for Imitation and print progress
product_launches['Imitation'] = 0
for index, row in product_launches.iterrows():
    imitation_count = calculate_imitation(row['Brewer ID'], row['Beer Style'], row['Time'], product_launches)
    product_launches.at[index, 'Imitation'] = 1 if imitation_count > 0 else 0
    if (index + 1) % print_interval == 0:
        print(f"Progress: {index + 1}/{total_rows}")

# Merge the Imitation results back into df3
df3 = df3.merge(product_launches[['Beer Style', 'Time', 'Brewer ID', 'Imitation']], 
                on=['Beer Style', 'Time', 'Brewer ID'], 
                how='left')

print("column Imitation was sucesfully created")
print(len(df3))
print(df3.head())

column Imitation was sucesfully created
1000
   Brewer ID  Unnamed: 0       Time                    Beer Style  \
0       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
2       8481           2  5-05-2000             Bohemian Pilsener   
3       8481           3  4-09-2006                        Kölsch   
4       8481           4  4-09-2006                        Kölsch   

   Rating First Digit  Review - Overall Beer ID  ABV  NewProduct  \
0                   1              0.65   63836  5.4           1   
1                   1              0.65   63836  5.4           1   
2                   1              0.70   71716    5           1   
3                   8              0.40   64125  5.4           1   
4                   8              0.40   64125  5.4           1   

  FirstReviewTime  NewMarket  NewCategory  Imitation  
0       4-09-2006          1            1        0.0  
1       0-09-2006    

In [403]:
print(len(df3))
print(df3.head())

1000
   Brewer ID  Unnamed: 0       Time                    Beer Style  \
0       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
2       8481           2  5-05-2000             Bohemian Pilsener   
3       8481           3  4-09-2006                        Kölsch   
4       8481           4  4-09-2006                        Kölsch   

   Rating First Digit  Review - Overall Beer ID  ABV  NewProduct  \
0                   1              0.65   63836  5.4           1   
1                   1              0.65   63836  5.4           1   
2                   1              0.70   71716    5           1   
3                   8              0.40   64125  5.4           1   
4                   8              0.40   64125  5.4           1   

  FirstReviewTime  NewMarket  NewCategory  Imitation  
0       4-09-2006          1            1        0.0  
1       0-09-2006          1            0        0.0  
2     

In [404]:
########## NegativePerformanceD

import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import math

# Assuming df3 is your existing DataFrame

# Filter df3 for new market entries
new_market_entries = df3[df3['NewMarket'] > 0]

# Group new market entries by 'Beer Style', 'Time', and 'Brewer ID'
product_launches = new_market_entries.groupby(['Beer Style', 'Time', 'Brewer ID']).size().reset_index(name='NewMarketEntryCount')

# Function to find competitors
def find_competitors(focal_brewer_id, beer_style, df):
    # Filter for the given beer style
    style_df = df[df['Beer Style'] == beer_style]

    # Find the first-digit rating of the focal firm's product
    focal_rating = style_df[style_df['Brewer ID'] == focal_brewer_id]['Rating First Digit'].iloc[0]

    # Find competitors: different brewer, same first-digit rating
    competitors = style_df[
        (style_df['Brewer ID'] != focal_brewer_id) &
        (style_df['Rating First Digit'] == focal_rating)
    ]['Brewer ID'].unique()

    return competitors

# Function to check if rating has declined
def has_rating_declined(brewer_id, beer_style, time, df):
    # Convert the 'Time' string to a datetime object
    time_obj = datetime.strptime(time, '%w-%m-%Y')

    # Define the time window: 3 weeks from the given date
    time_window_start = time_obj - relativedelta(weeks=3)
    time_window_end = time_obj

    # Filter for the specific brewer, beer style, and time window
    ratings_df = df[
        (df['Brewer ID'] == brewer_id) &
        (df['Beer Style'] == beer_style) &
        (df['Time'] >= time_window_start.strftime('%w-%m-%Y')) &
        (df['Time'] <= time_window_end.strftime('%w-%m-%Y'))
    ]

    # Check if ratings declined
    if not ratings_df.empty:
        sorted_ratings = ratings_df.sort_values(by='Time')
        start_rating = sorted_ratings['Rating First Digit'].iloc[0]
        end_rating = sorted_ratings['Rating First Digit'].iloc[-1]
        return end_rating < start_rating
    else:
        return False

# Function to check negative performance
def check_negative_performance(row, df):
    # Identify competitors
    competitors = find_competitors(row['Brewer ID'], row['Beer Style'], df)

    # Check if the rating declined over the last three weeks
    has_declined = has_rating_declined(row['Brewer ID'], row['Beer Style'], row['Time'], df)

    if len(competitors) > 0 and has_declined:
        return 1
    else:
        return 0

# Apply the function in batches of 10 rows
total_rows = len(df3)
chunks = math.ceil(total_rows / 10)

# Apply the function to each row in df3 and print progress every 10 rows
total_launches = len(product_launches)
for i, row in product_launches.iterrows():
    # Find the corresponding row(s) in df3
    df3_indices = df3[(df3['Brewer ID'] == row['Brewer ID']) & 
                      (df3['Beer Style'] == row['Beer Style']) & 
                      (df3['Time'] == row['Time'])].index

    # Calculate NegativePerformanceD for each corresponding row in df3
    for idx in df3_indices:
        df3.at[idx, 'NegativePerformanceD'] = check_negative_performance(df3.iloc[idx], df3)

    # Print progress every 10 rows
    if (i+1) % 10 == 0:
        print(f"Processed {i+1} product launches out of {total_launches}")



Processed 10 product launches out of 58
Processed 20 product launches out of 58
Processed 30 product launches out of 58
Processed 40 product launches out of 58
Processed 50 product launches out of 58


In [405]:
print(len(df3))
print(df3.head())
print(df3['NegativePerformanceD'].max())

1000
   Brewer ID  Unnamed: 0       Time                    Beer Style  \
0       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
2       8481           2  5-05-2000             Bohemian Pilsener   
3       8481           3  4-09-2006                        Kölsch   
4       8481           4  4-09-2006                        Kölsch   

   Rating First Digit  Review - Overall Beer ID  ABV  NewProduct  \
0                   1              0.65   63836  5.4           1   
1                   1              0.65   63836  5.4           1   
2                   1              0.70   71716    5           1   
3                   8              0.40   64125  5.4           1   
4                   8              0.40   64125  5.4           1   

  FirstReviewTime  NewMarket  NewCategory  Imitation  NegativePerformanceD  
0       4-09-2006          1            1        0.0                   0.0  
1       0-09-2006

In [406]:
########## Negative performance - continuous

def competitors_new_market_avg_rating(brewer_id, beer_style, time, df):
    competitors = find_competitors(brewer_id, beer_style, df)
    if len(competitors) > 0:
        competitor_new_market_ratings = df[
            (df['Brewer ID'].isin(competitors)) &
            (df['Beer Style'] == beer_style) &
            (df['Time'] == time) &
            (df['NewMarket'] > 0)
        ]['Review - Overall']
        if not competitor_new_market_ratings.empty:
            return competitor_new_market_ratings.mean()
    return None


def calculate_negative_performance_c(row, df):
    competitors_new_market_rating = competitors_new_market_avg_rating(row['Brewer ID'], row['Beer Style'], row['Time'], df)
    if competitors_new_market_rating is not None and competitors_new_market_rating != 0:
        return (row['Review - Overall'] - competitors_new_market_rating) / competitors_new_market_rating
    else:
        return 0  # Assign 0 if conditions are not met


total_launches = len(product_launches)
for i, row in product_launches.iterrows():
    # Find the corresponding row(s) in df3
    df3_indices = df3[(df3['Brewer ID'] == row['Brewer ID']) & 
                      (df3['Beer Style'] == row['Beer Style']) & 
                      (df3['Time'] == row['Time'])].index

    # Calculate NegativePerformanceC for each corresponding row in df3
    for idx in df3_indices:
        df3.at[idx, 'NegativePerformanceC'] = calculate_negative_performance_c(df3.iloc[idx], df3)

    # Print progress every 10 rows
    if (i+1) % 10 == 0:
        print(f"Processed {i+1} product launches out of {total_launches}")




Processed 10 product launches out of 58
Processed 20 product launches out of 58
Processed 30 product launches out of 58
Processed 40 product launches out of 58
Processed 50 product launches out of 58


In [407]:
print(len(df3))
print(df3.head())
print(df3['NegativePerformanceC'].max())

1000
   Brewer ID  Unnamed: 0       Time                    Beer Style  \
0       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
2       8481           2  5-05-2000             Bohemian Pilsener   
3       8481           3  4-09-2006                        Kölsch   
4       8481           4  4-09-2006                        Kölsch   

   Rating First Digit  Review - Overall Beer ID  ABV  NewProduct  \
0                   1              0.65   63836  5.4           1   
1                   1              0.65   63836  5.4           1   
2                   1              0.70   71716    5           1   
3                   8              0.40   64125  5.4           1   
4                   8              0.40   64125  5.4           1   

  FirstReviewTime  NewMarket  NewCategory  Imitation  NegativePerformanceD  \
0       4-09-2006          1            1        0.0                   0.0   
1       0-09-20

In [408]:
########## CompetitorPosition

def compare_competitor_review(focal_brewer_id, beer_style, time, df):
    focal_review = df[(df['Brewer ID'] == focal_brewer_id) &
                      (df['Beer Style'] == beer_style) &
                      (df['Time'] == time)]['Review - Overall'].iloc[0]

    competitors = find_competitors(focal_brewer_id, beer_style, df)
    if len(competitors) > 0:
        competitors_reviews = df[
            (df['Brewer ID'].isin(competitors)) &
            (df['Beer Style'] == beer_style) &
            (df['Time'] == time)
        ]['Review - Overall']

        if not competitors_reviews.empty:
            # Check if any competitor has a higher review
            return 1 if any(competitors_reviews > focal_review) else 0
    return 0



total_launches = len(product_launches)
for i, row in product_launches.iterrows():
    # Find the corresponding row(s) in df3
    df3_indices = df3[(df3['Brewer ID'] == row['Brewer ID']) &
                      (df3['Beer Style'] == row['Beer Style']) &
                      (df3['Time'] == row['Time'])].index

    # Calculate CompetitorPosition for each corresponding row in df3
    for idx in df3_indices:
        df3.at[idx, 'CompetitorPosition'] = compare_competitor_review(df3.iloc[idx]['Brewer ID'], df3.iloc[idx]['Beer Style'], df3.iloc[idx]['Time'], df3)

    # Print progress every 10 rows
    if (i+1) % 10 == 0:
        print(f"Processed {i+1} product launches out of {total_launches}")


Processed 10 product launches out of 58
Processed 20 product launches out of 58
Processed 30 product launches out of 58
Processed 40 product launches out of 58
Processed 50 product launches out of 58


In [409]:
print(len(df3))
print(df3.head())
print(df3['CompetitorPosition'].max())

1000
   Brewer ID  Unnamed: 0       Time                    Beer Style  \
0       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
2       8481           2  5-05-2000             Bohemian Pilsener   
3       8481           3  4-09-2006                        Kölsch   
4       8481           4  4-09-2006                        Kölsch   

   Rating First Digit  Review - Overall Beer ID  ABV  NewProduct  \
0                   1              0.65   63836  5.4           1   
1                   1              0.65   63836  5.4           1   
2                   1              0.70   71716    5           1   
3                   8              0.40   64125  5.4           1   
4                   8              0.40   64125  5.4           1   

  FirstReviewTime  NewMarket  NewCategory  Imitation  NegativePerformanceD  \
0       4-09-2006          1            1        0.0                   0.0   
1       0-09-20

In [410]:
########## MRank

# Convert 'Time' to datetime and extract the month and year
df3['MonthYear'] = pd.to_datetime(df3['Time'], format='%w-%m-%Y').dt.to_period('M')

# Rank products within each group (e.g., 'Beer Style') for each month
df3['MonthlyRank'] = df3.groupby(['Beer Style', 'MonthYear'])['Review - Overall'].rank("average", ascending=False)

# Group by the relevant identifier (e.g., 'Brewer ID') and calculate the average rank
MRank = df3.groupby('Brewer ID')['MonthlyRank'].mean().reset_index(name='MRank')

# Merge this back into the original df3 if needed
df3 = df3.merge(MRank, on='Brewer ID', how='left')


In [411]:
print(len(df3))
print(df3.head())
print(df3['MRank'].max())

1000
   Brewer ID  Unnamed: 0       Time                    Beer Style  \
0       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
2       8481           2  5-05-2000             Bohemian Pilsener   
3       8481           3  4-09-2006                        Kölsch   
4       8481           4  4-09-2006                        Kölsch   

   Rating First Digit  Review - Overall Beer ID  ABV  NewProduct  \
0                   1              0.65   63836  5.4           1   
1                   1              0.65   63836  5.4           1   
2                   1              0.70   71716    5           1   
3                   8              0.40   64125  5.4           1   
4                   8              0.40   64125  5.4           1   

  FirstReviewTime  NewMarket  NewCategory  Imitation  NegativePerformanceD  \
0       4-09-2006          1            1        0.0                   0.0   
1       0-09-20

In [412]:
########## SDRank

# Convert 'Time' to datetime and extract the month and year
df3['MonthYear'] = pd.to_datetime(df3['Time'], format='%w-%m-%Y').dt.to_period('M')

# Rank products within each group (e.g., 'Beer Style') for each month based on 'Review - Overall'
df3['MonthlyRank'] = df3.groupby(['Beer Style', 'MonthYear'])['Review - Overall'].rank("average", ascending=False)

# Group by the relevant identifier (e.g., 'Brewer ID') and calculate the standard deviation of the rank
SDRank = df3.groupby('Brewer ID')['MonthlyRank'].std().reset_index(name='SDRank')

# Merge this back into the original df3
df3 = df3.merge(SDRank, on='Brewer ID', how='left')


In [413]:
print(len(df3))
print(df3.head())
print(df3['SDRank'].max())

1000
   Brewer ID  Unnamed: 0       Time                    Beer Style  \
0       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
2       8481           2  5-05-2000             Bohemian Pilsener   
3       8481           3  4-09-2006                        Kölsch   
4       8481           4  4-09-2006                        Kölsch   

   Rating First Digit  Review - Overall Beer ID  ABV  NewProduct  \
0                   1              0.65   63836  5.4           1   
1                   1              0.65   63836  5.4           1   
2                   1              0.70   71716    5           1   
3                   8              0.40   64125  5.4           1   
4                   8              0.40   64125  5.4           1   

  FirstReviewTime  NewMarket  NewCategory  Imitation  NegativePerformanceD  \
0       4-09-2006          1            1        0.0                   0.0   
1       0-09-20

In [414]:
########## Competition

# Assuming 'Beer Style' is the product category

# Group by 'Beer Style' and 'MonthYear', and count unique 'Brewer ID'
competition_counts = df3.groupby(['Beer Style', 'MonthYear'])['Brewer ID'].nunique().reset_index(name='Competition')

# Merge this count back into the original df3
df3 = df3.merge(competition_counts, on=['Beer Style', 'MonthYear'], how='left')



In [415]:
print(len(df3))
print(df3.head())
print(df3['Competition'].max())

1000
   Brewer ID  Unnamed: 0       Time                    Beer Style  \
0       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
2       8481           2  5-05-2000             Bohemian Pilsener   
3       8481           3  4-09-2006                        Kölsch   
4       8481           4  4-09-2006                        Kölsch   

   Rating First Digit  Review - Overall Beer ID  ABV  NewProduct  \
0                   1              0.65   63836  5.4           1   
1                   1              0.65   63836  5.4           1   
2                   1              0.70   71716    5           1   
3                   8              0.40   64125  5.4           1   
4                   8              0.40   64125  5.4           1   

  FirstReviewTime  ...  NewCategory  Imitation  NegativePerformanceD  \
0       4-09-2006  ...            1        0.0                   0.0   
1       0-09-2006  ...     

In [416]:
########## 

# Convert 'Time' to datetime
df3['Date'] = pd.to_datetime(df3['Time'], format='%w-%m-%Y')

# For each row, calculate the number of products receiving reviews in the past 12 months
def count_products_last_12_months(row, df):
    start_date = row['Date'] - pd.DateOffset(months=12)
    end_date = row['Date']
    return df[(df['Brewer ID'] == row['Brewer ID']) & 
              (df['Date'] >= start_date) & 
              (df['Date'] <= end_date)]['Beer ID'].nunique()

df3['Portfolio'] = df3.apply(lambda row: count_products_last_12_months(row, df3), axis=1)


In [417]:
import re  # Import the regular expressions module

def convert_abv_to_numeric(abv_string):
    try:
        # Split the string by any non-numeric character and filter out empty strings
        abv_values = [float(value) for value in re.split('[^0-9.]+', abv_string) if value]
        return sum(abv_values) / len(abv_values) if abv_values else None
    except ValueError:
        return None

# Apply this conversion to the 'ABV' column
df3['ABV'] = df3['ABV'].apply(convert_abv_to_numeric)

# Function to calculate the average ABV in the past 12 months
def average_abv_last_12_months(row, df):
    start_date = row['Date'] - pd.DateOffset(months=12)
    end_date = row['Date']
    relevant_data = df[(df['Brewer ID'] == row['Brewer ID']) & 
                       (df['Date'] >= start_date) & 
                       (df['Date'] <= end_date)]
    # Calculate the average ABV
    if not relevant_data.empty:
        return relevant_data['ABV'].mean()
    else:
        return None

# Apply the function to df3
df3['ABV'] = df3.apply(lambda row: average_abv_last_12_months(row, df3), axis=1)


In [418]:
#print(len(df3))
#print(df3.head())
print(df3['ABV'].max())

7.0


In [419]:
########## Diversification

def count_unique_categories_last_12_months(row, df):
    start_date = row['Date'] - pd.DateOffset(months=12)
    end_date = row['Date']
    relevant_data = df[(df['Brewer ID'] == row['Brewer ID']) & 
                       (df['Date'] >= start_date) & 
                       (df['Date'] <= end_date)]
    # Count unique product categories
    return relevant_data['Beer Style'].nunique()

df3['Diversification'] = df3.apply(lambda row: count_unique_categories_last_12_months(row, df3), axis=1)



In [420]:
#print(len(df3))
#print(df3.head())
print(df3['Diversification'].max())

14


In [421]:
########## Tenure

first_review_dates = df3.groupby('Brewer ID')['Date'].min().reset_index()
first_review_dates.rename(columns={'Date': 'FirstReviewDate'}, inplace=True)

# Merge the first review dates with df3
df3 = df3.merge(first_review_dates, on='Brewer ID', how='left')

# Calculate Tenure as the difference between the 'Date' and 'FirstReviewDate'
df3['Tenure'] = (df3['Date'] - df3['FirstReviewDate']).dt.days



In [422]:
#print(len(df3))
#print(df3.head())
print(df3['Tenure'].max())

3106


In [423]:
########## Experience

product_launches['Date'] = pd.to_datetime(product_launches['Time'], format='%w-%m-%Y')

# Sort by 'Brewer ID' and 'Date'
product_launches = product_launches.sort_values(by=['Brewer ID', 'Date'])

# Count cumulative launches
product_launches['CumulativeLaunches'] = product_launches.groupby('Brewer ID').cumcount() + 1

# Merge cumulative launch counts into df3
df3 = df3.merge(product_launches[['Brewer ID', 'Date', 'CumulativeLaunches']], on=['Brewer ID', 'Date'], how='left')

# If there are any NaN values (i.e., dates in df3 without a corresponding launch in product_launches), fill them with the previous valid value
df3['CumulativeLaunches'] = df3.groupby('Brewer ID')['CumulativeLaunches'].fillna(method='ffill')

# Rename the column to 'Experience'
df3.rename(columns={'CumulativeLaunches': 'Experience'}, inplace=True)

/var/folders/88/g84kk_9n2498jwvfh8mnxj8w0000gn/T/ipykernel_96284/3621156880.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df3['CumulativeLaunches'] = df3.groupby('Brewer ID')['CumulativeLaunches'].fillna(method='ffill')


In [424]:
#print(len(df3))
#print(df3.head())
print(df3['Experience'].max())

47.0


In [425]:
########## Popularity

df3 = df3.sort_values(by=['Brewer ID', 'Date'])
df3['Popularity'] = df3.groupby('Brewer ID').cumcount() + 1




In [426]:
#print(len(df3))
#print(df3.head())
print(df3['Popularity'].max())

1416


In [427]:
df3 = df3[['Brewer ID', 'Time', 'NewProduct', 'NewMarket', 'NewCategory', 
           'Imitation', 'NegativePerformanceD', 'NegativePerformanceC', 
           'CompetitorPosition', 'MonthlyRank', 'MRank', 'SDRank', 
           'Competition', 'Portfolio', 'Diversification', 'FirstReviewDate', 
           'Tenure', 'Experience', 'Popularity']]


In [430]:
excel_file_path = '/Users/bruno/Documents/Statistical_Analyst_Products/results_1000.xlsx'

# Export the DataFrame to Excel
df3.to_excel(excel_file_path, index=False)  # Set index=False to exclude the index column

print(f'DataFrame has been exported to {excel_file_path}')

DataFrame has been exported to /Users/bruno/Documents/Statistical_Analyst_Products/results_1000.xlsx
